### Dashboard to explore glider data

In [ ]:
import holoviews as hv
import hvplot.xarray 
import numpy as np
import panel as pn
import xarray as xr

In [ ]:
# Holoviews extensions
hv.extension('bokeh')
pn.extension()

### Load in surface variables

In [ ]:
# Open data

# open Surface fields
fname_SSH = "./data/SSH_sogos.nc"
fname_FSLE = "./data/FSLE_sogos.nc"

ds_ssh = xr.open_dataset(fname_SSH)
ds_fsle = xr.open_dataset(fname_FSLE)

# Create variables for vectors
ds_ssh['mag'] = np.sqrt(ds_ssh.ugos**2 + ds_ssh.vgos**2)
ds_ssh['angle'] = (np.pi/2.) - np.arctan2(ds_ssh.ugos/ds_ssh['mag'], 
                                          ds_ssh.vgos/ds_ssh['mag'])

In [ ]:
days = ds_ssh.time - np.datetime64('2019-01-01')
ds_ssh['days'] = (days / np.timedelta64(1, 'D'))

days = ds_fsle.time - np.datetime64('2019-01-01')
ds_fsle['days'] = (days / np.timedelta64(1, 'D'))

### Load glider data

In [ ]:
# open glider files
glid_folder = './data/'
ds_659 = xr.load_dataset(glid_folder + 'sg659/CTD_659.nc')
ds_660 = xr.load_dataset(glid_folder + 'sg660/CTD_660.nc')

# easier to work with a days variable that is a float rather than datenum
days = ds_659.time - np.datetime64('2019-01-01')
ds_659['days'] = (days / np.timedelta64(1, 'D'))

days = ds_660.time - np.datetime64('2019-01-01')
ds_660['days'] = (days / np.timedelta64(1, 'D'))

# Group and average by dives so that plotting of positions is fast
ds_659_av = ds_659.groupby('dives').mean()
ds_660_av = ds_660.groupby('dives').mean()

ds_659_av = ds_659_av.drop('dives')
ds_660_av = ds_660_av.drop('dives')

In [ ]:
# convert from point data to gridded data, can take some time
ds_659_grid = xr.load_dataset(glid_folder + '659_grid.nc')
ds_660_grid = xr.load_dataset(glid_folder + '660_grid.nc')

### Surface plots 

In [ ]:
# Create Widgets
surf_var_select = pn.widgets.Select(name='Surface Field',
                    options = ['SSH','SSHA','FSLE'])

time_slider = pn.widgets.IntSlider(start=int(ds_ssh.days.min().values),
                                end = int(ds_ssh.days.max().values))

time_range_slider = pn.widgets.IntRangeSlider(name='Year Day Range',start=int(ds_ssh.days.min().values),
                                end = int(ds_ssh.days.max().values), 
                                value=(int(ds_ssh.days.min().values), int(ds_ssh.days.min().values)+10))

glider_select = pn.widgets.Select(name='Glider #', options=['659', '660'])

In [ ]:
glider_var_select = pn.widgets.Select(name='Glider Variable', options=['temperature', 'salinity','spice'])

In [ ]:
# generate the maps
@pn.depends(surf_var_select, time_range_slider, glider_select)
def combined_map(var_sel, time_rng, glid_sel):
#@pn.depends(var_sel = surf_var_select.param.value, 
#            time_rng = time_range_slider.param.value, 
#            glid_sel = glider_select.param.value)
#def combined_map(var_sel, time_rng, glid_sel):
    surf_var_plot = plot_surface_var(time_rng[1], var_sel)
    glid_track_plot =  plot_glider_tracks(time_rng, glid_sel)
   
    final_plot =  (surf_var_plot*glid_track_plot).opts(frame_width=400, frame_height=400, 
                                                       xlim=(28,40), ylim=(-55,-49)
                                               ).opts(
                                                     title =str(ds_ssh.time.where(ds_ssh.days==time_rng[1], drop=True).squeeze('time').values)[0:10])
    
    return final_plot
    
    
    
# generate the right surface contourf/image  
#@pn.depends(time_sel = time_slider.param.value, var_sel=surf_var_select.param.value )
def plot_surface_var(time_sel, var_sel):
    vec_plot = ds_ssh.where(ds_ssh.days==time_sel,drop=True).squeeze('time').hvplot.vectorfield(
                                      x='longitude', y='latitude', angle='angle', mag='mag',
                                       hover=False).opts(magnitude='mag')
    
    if var_sel == 'SSH':
        plot_gen = ds_ssh['adt'].where(ds_ssh.days==time_sel,drop=True
                                      ).squeeze('time').hvplot.contourf(levels=np.linspace(-1.2,0,21)
                                                                        ).opts(clim=(-1,0), 
                                                                               cmap='RdBu_r',
                                                                               clabel='SSH')
        
    elif var_sel =='SSHA':
        plot_gen = ds_ssh['sla'].where(ds_ssh.days==time_sel,drop=True
                                      ).squeeze('time').hvplot.contourf(levels=np.linspace(-.4,0.6,21), 
                                                                     ).opts(clim=(-0.3,0.3),
                                                                            cmap='RdBu_r',
                                                                            clabel='SSHA')
    
    elif var_sel == 'FSLE':
        plot_gen = ds_fsle['fsle_max'].where(ds_ssh.days==time_sel,drop=True
                                            ).squeeze('time').hvplot.image().opts(clim=(-0.6,0),
                                                                                  cmap='Blues_r',
                                                                                  clabel='FSLE')
        
    composite = (plot_gen*vec_plot)
                            #str(ds_ssh.time.where(ds_ssh.days==150, drop=True).squeeze('time').values)[0:10] )
    
    return composite


# Generate the glider tracks
#@pn.depends(time_rng = time_range_slider.param.value, glid_sel = glider_select.param.value)
def plot_glider_tracks(time_rng, glid_sel="659"):
    
    width_659=2
    width_660=2
    
    if glid_sel=="659":
        width_659 = 4.
    elif glid_sel=="660":
        width_660=4.
        
    
    plot_660 = ds_660_av.where(np.logical_and(ds_660_av.days>time_rng[0], 
                                              ds_660_av.days<time_rng[1]), drop=True 
                          ).hvplot(x='longitude', y='latitude', 
                                   hover=True, hover_cols=['days'], 
                                   line_width=width_660, line_color="orange" )
    
    plot_659 = ds_659_av.where(np.logical_and(ds_659_av.days>time_rng[0], 
                                              ds_659_av.days<time_rng[1]), drop=True 
                          ).hvplot(x='longitude', y='latitude', hover=True, 
                                   hover_cols=['days'],
                                  line_width=width_659, line_color="green")
    
    composite = (plot_660 * plot_659)
    
    return composite

In [ ]:
@pn.depends(glider_select, glider_var_select)
def plot_glider(glid_num, glid_var):
    
    bin_range = ()
    cmap_sel = 'RdBu_r'
        
    if glid_var=='spice':
        bin_range = (-0.7, 0.02)
        cmap_sel = 'RdBu_r'
    
    elif glid_var=='salinity':
        bin_range=(33.75, 35)
        cmap_sel = 'RdBu_r'
        
    elif glid_var=='temperature':
        bin_range=(0,4)
        cmap_sel='RdBu_r'
        
    
    if glid_num=='659':
       
        plot_gen_image = ds_659_grid[glid_var].hvplot.image(cmap=cmap_sel, 
                                                flip_yaxis=True, clim=bin_range).apply.opts(
                                                xlim = time_range_slider).hist(bin_range = bin_range)
        plot_gen_contour = ds_659_grid['potdens'].hvplot.contour(flip_yaxis=True, 
                                                                 levels=np.linspace(1026.8, 1027.8,11)
                                                                 ).apply.opts(xlim = time_range_slider)
        
        plot_gen = plot_gen_image*plot_gen_contour
        
    elif glid_num=='660':
        
        plot_gen_image = ds_660_grid[glid_var].hvplot.image(cmap=cmap_sel, 
                                                flip_yaxis=True, clim=bin_range).apply.opts(
                                                xlim = time_range_slider).hist(bin_range = bin_range)
        plot_gen_contour = ds_660_grid['potdens'].hvplot.contour(flip_yaxis=True, 
                                                                 levels=np.linspace(1026.8, 1027.8,11)
                                                                ).apply.opts(xlim = time_range_slider)
        
        plot_gen = plot_gen_image*plot_gen_contour
        
    return plot_gen


In [ ]:
Surf = pn.Column(pn.Row(
    pn.Column(pn.panel('# Southern Ocean Glider Observations of the Submesoscales'),
              pn.panel('## Glider Explorer'),
              pn.WidgetBox(surf_var_select, time_range_slider, glider_select, glider_var_select)), 
       combined_map), plot_glider)

In [ ]:
# working locally
#Surf.show()

# mybinder.org app
Surf.servable()